In [2]:
#导入包
import pandas as pd

import jieba
import jieba.analyse as ana
import jieba.posseg as psg
import nltk
from nltk import FreqDist
from sklearn.model_selection import train_test_split
from gensim.models.word2vec import Word2Vec
import joblib
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from tqdm import tqdm
import os
import pymysql
import re
from itertools import chain
import json

In [12]:
#根据 用户id，微博内容，情绪 对数据进行去重，
targetRoot = r'D:\数据挖掘比赛\选题1\去重数据\\'
sourceRoot = r'D:\数据挖掘比赛\选题1\微博'
files = None
for root,dirs,fs in os.walk(sourceRoot):
    files = fs
    break
for file in files:
    fileAbsolutePath = sourceRoot+'\\'+file
    data = pd.read_csv(fileAbsolutePath,encoding='gb18030')
    NoDuplicateData = data.drop_duplicates(["全文内容","微博情绪","MD5-作者ID"],keep="first")
    NoDuplicateData.to_csv(targetRoot+file,encoding="gb18030")
    # print("数据个数：",len(data),"->",len(NoDuplicateData))

数据个数： 221902 -> 218591
数据个数： 392935 -> 390198
数据个数： 689909 -> 639337
数据个数： 360625 -> 339905
数据个数： 1830 -> 1807
数据个数： 39914 -> 39502
数据个数： 17314 -> 14789
数据个数： 87024 -> 85933
数据个数： 749899 -> 393161
数据个数： 749917 -> 368273
数据个数： 519264 -> 230253
数据个数： 76049 -> 75320
数据个数： 3654 -> 3584
数据个数： 2426 -> 2284
数据个数： 2226 -> 2219
数据个数： 16994 -> 16803
数据个数： 698270 -> 673898
数据个数： 491431 -> 473720
数据个数： 136483 -> 128039


In [13]:
#把所有数据按照情绪进行分类,并进行分词和词性标注，存入csv文件中
#【全文内容，情绪，jieba分词，jieba词性标注】
emotionRoot = r'D:\数据挖掘比赛\选题1\情绪分类数据'
NoDuplicateRoot = r'D:\数据挖掘比赛\选题1\去重数据'
files = None
indexlist = [1,1,1,1,1,1]
leng = 0    #记录去重后数据长度
for root,dirs,fs in os.walk(NoDuplicateRoot):
    files = fs
    break
for id,file in enumerate(files):
    fileAbsolutePath = NoDuplicateRoot+'\\'+file
    data = pd.read_csv(fileAbsolutePath,encoding='gb18030')
    Gdata = data.groupby(by='微博情绪')  # 根据微博情绪进行分组
    PosData = Gdata.get_group('喜悦').loc[:, '全文内容']   #提取出对应内容
    NegData = Gdata.get_group('悲伤').loc[:, '全文内容']
    angerData = Gdata.get_group('愤怒').loc[:, '全文内容']
    middleData = Gdata.get_group('中性').loc[:, '全文内容']
    amazeData = Gdata.get_group('惊奇').loc[:, '全文内容']
    fearData = Gdata.get_group('恐惧').loc[:, '全文内容']
    emotionData = [PosData,NegData,angerData,middleData,amazeData,fearData]
    emotion = ["喜悦","悲伤","愤怒","中性","惊奇","恐惧"]

    #emotionData.append()
    for idx,emotionList in enumerate(emotionData):
        emotionList = emotionList.to_frame()    ##将series变成dataframe类型
        emotionList = emotionList.drop_duplicates()     ##去掉完全重复的数据
        leng += len(emotionList)
        emotionList["分词"] = emotionList["全文内容"].apply(jieba.lcut)
        emotionList["词性标注"] = emotionList["全文内容"].apply(psg.cut)
        a = indexlist[idx] + len(emotionList)       #让存入csv文件的序号有序，从1开始
        emotionList.index = np.arange(indexlist[idx],a)
        indexlist[idx] = a
        if(id == 0):
            emotionList.to_csv(emotionRoot+'\\'+emotion[idx]+'.csv',encoding="gb18030",mode="a")
        else:
            emotionList.to_csv(emotionRoot+'\\'+emotion[idx]+'.csv',encoding="gb18030",mode="a",header=False)
    print("数据长度：",len(data),"->",leng)
    print("分类长度索引",indexlist)

数据长度： 218591 -> 57353
分类长度索引 [21040, 6994, 6825, 17699, 3613, 1188]
数据长度： 390198 -> 258171
分类长度索引 [95725, 43556, 54814, 51460, 10467, 2155]
数据长度： 639337 -> 620070
分类长度索引 [314250, 121722, 72920, 90716, 14195, 6273]
数据长度： 339905 -> 827273
分类长度索引 [440023, 166606, 82783, 112931, 16238, 8698]
数据长度： 1807 -> 828320
分类长度索引 [440258, 166634, 83061, 113412, 16239, 8722]
数据长度： 39502 -> 850600
分类长度索引 [446893, 169877, 89222, 118681, 16656, 9277]
数据长度： 14789 -> 858058
分类长度索引 [447967, 170316, 90600, 123065, 16767, 9349]
数据长度： 85933 -> 904044
分类长度索引 [455502, 172553, 106129, 139377, 20386, 10103]
数据长度： 393161 -> 1027372
分类长度索引 [546102, 199037, 107813, 143448, 20541, 10437]
数据长度： 368273 -> 1136122
分类长度索引 [624014, 224431, 109233, 147148, 20660, 10642]
数据长度： 230253 -> 1199915
分类长度索引 [666518, 242848, 109885, 149239, 20710, 10721]
数据长度： 75320 -> 1238742
分类长度索引 [686352, 251257, 114463, 153754, 20990, 11932]
数据长度： 3584 -> 1240652
分类长度索引 [686621, 251300, 114575, 155102, 21060, 12000]
数据长度： 2284 -> 1241794
分类长度索

In [10]:
# -*- coding = utf-8 -*-
#对词性标注数据进行词频统计（利用正则表达式去除非中文符号），放进数据库表格中，【词，词性，词频】
emotionRoot = r'D:\数据挖掘比赛\选题1\情绪分类数据'
for root,dirs,fs in os.walk(emotionRoot):
    files = fs
    break
for id,file in enumerate(files):
    fileAbsolutePath = emotionRoot+'\\'+file
    data = pd.read_csv('D:\数据挖掘比赛\选题1\情绪分类数据\恐惧.csv',encoding='gb18030')
    # print(type(data))
    # print(data["分词"][0])
    # for i in data["分词"][0]:
    #     print(i)
    # print(type(data))
    # data = ['我们']
    # print()
    print(len(data["分词"][0]))
    pattern = "[\u4e00-\u9fa5]+"    #中文正则表达式
    regex = re.compile(pattern)     #正则对象
    print(' '.join(data["分词"][0]))
    print(json.dumps(regex.findall(' '.join(data["分词"][0]))))
    # data["分词"] = data["分词"].apply(lambda x: regex.findall(' '.join(x)))

    # print(data["分词"])#
    # wordNum = []
    # # for i in data["分词"]:
    # #     wordNum = wordNum+i
    # datalist = data["分词"].values
    # datalist = list(chain.from_iterable(datalist))
    # datalist = list(chain.from_iterable(datalist))
    # fdist = FreqDist(datalist)
    # dbConnect = pymysql.connect(host='182.61.54.181',port=3306,db='WRD',user='yonYi',password='YonYi12!')
    # dbCursor = dbConnect.cursor()   #游标对象
    # table_name = file.strip(".csv") + "jieba分词词频"
    # dbCursor.execute("DROP TABLE IF EXISTS %s"%(table_name))
    # # 使用预处理语句创建表
    # sql = """CREATE TABLE  %s(
    #         word  CHAR(100),
    #         num INT)
    #         """%(table_name)#表名为变量
    # dbCursor.execute(sql)
    # for word,num in fdist.most_common():
    #     sql1 = '''
    #         insert into  %s(
    #             word,num
    #         )
    #         values("%s","%d")
    #     '''%(table_name,word,num)    #values里面的%s尽量用双引号
    #     dbCursor.execute(sql1)
    #     # # sql = "SELECT * FROM table_name ORDER BY name DESC"
    #     # sql2 = '''
    #     #     SELECT * FROM %s ORDER BY num DESC
    #     # '''%(table_name)    #values里面的%s尽量用双引号
    #     # dbCursor.execute(sql2)
    #     dbConnect.commit()      #提交
    # dbCursor.close()
    # dbConnect.close()
    # print(file.strip(".csv") + "表已创建！")

264
[ ' 明 年 ' ,   ' 怕 ' ,   ' 不 是 ' ,   ' 要 ' ,   ' 破 ' ,   ' 五 千 ' ,   ' 亿 ' ,   '   ' ,   ' 【 ' ,   ' 原 微 博 ' ,   ' 】 ' ,   '   ' ,   ' @ ' ,   ' 天 下 网 ' ,   ' 商 ' ,   '   ' ,   ' 4 9 8 2 ' ,   ' 亿 ' ,   ' ！ ' ,   ' 截 至 ' ,   ' 1 1 ' ,   ' 月 ' ,   ' 1 1 ' ,   ' 日 ' ,   ' 2 4 ' ,   ' 时 ' ,   ' ， ' ,   ' 2 0 2 0 ' ,   ' 年 天 ' ,   ' 猫 ' ,   ' 双 ' ,   ' 1 1 ' ,   ' 全 球 ' ,   ' 狂 欢 ' ,   ' 季 ' ,   ' 成 交 额 ' ,   ' 4 9 8 2 ' ,   ' 亿 元 ' ,   ' 。 ' ,   ' # ' ,   ' 天 猫 ' ,   ' 双 ' ,   ' 1 1 ' ,   ' # ' ,   '   ' ,   ' ? ' ,   ' ? ' ]
["\u660e", "\u5e74", "\u6015", "\u4e0d", "\u662f", "\u8981", "\u7834", "\u4e94", "\u5343", "\u4ebf", "\u539f", "\u5fae", "\u535a", "\u5929", "\u4e0b", "\u7f51", "\u5546", "\u4ebf", "\u622a", "\u81f3", "\u6708", "\u65e5", "\u65f6", "\u5e74", "\u5929", "\u732b", "\u53cc", "\u5168", "\u7403", "\u72c2", "\u6b22", "\u5b63", "\u6210", "\u4ea4", "\u989d", "\u4ebf", "\u5143", "\u5929", "\u732b", "\u53cc"]
264
[ ' 明 年 ' ,   ' 怕 ' ,   ' 不 是 ' ,   ' 要 ' ,   ' 破 ' ,   ' 五 千 '

In [ ]:
#jieba分词数据进行词频统计，放进数据库表格中。【词，词频】


In [ ]:
#取出词性标注数据，去除名词，标点符号等无用词，进行词的向量化

In [ ]:
#模型训练